In [ ]:
import pandas as pd

df = pd.read_csv("data/predictions.csv")
df.head(5)


: 

In [ ]:
import mlflow
import mlflow.pyfunc

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
print(df["predictions"].value_counts())

In [ ]:
model_uri = "models:/CreditCardFraudModel/Production"
loaded_model = mlflow.pyfunc.load_model(model_uri)
print("✅ 모델 불러오기 완료:", model_uri)

# X, y 분리 (predictions 컬럼은 예측값, Class가 있으면 실제 라벨)
X_new = df.drop(columns=["predictions"])
y_true = df["Class"] if "Class" in df.columns else None

preds = loaded_model.predict(X_new)

df["predictions"] = preds

# 3. 금융적 해석: 사기 거래 비율
fraud_ratio = (df["predictions"] == 1).mean() * 100
print(f"💳 전체 거래 중 사기 거래 예측 비율: {fraud_ratio:.3f}%")

# 금액대별 사기 거래 평균 금액 비교
fraud_amount_mean = df.loc[df["predictions"] == 1, "Amount"].mean()
normal_amount_mean = df.loc[df["predictions"] == 0, "Amount"].mean()

print(f"📊 사기 거래 평균 금액: {fraud_amount_mean:.2f}")
print(f"📊 정상 거래 평균 금액: {normal_amount_mean:.2f}")